In [68]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [74]:
SEASONS = list(range(2024,2025))

In [75]:
DATA_DIR = "data"
HTML_DIR = "HTML"
STANDINGS_DIR = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/standings'
SCORES_DIR = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/scores'

In [76]:
async def get_html(url, selector, sleep=5, retries = 5):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)  # Use asyncio.sleep() instead of time.sleep()

        
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [77]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_page = [f"https://www.basketball-reference.com{l}" for l in href]
    
    for url in standings_page:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        print(save_path)
        if os.path.exists(save_path) and save_path != "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/standings/NBA_2024_games-april.html":
            continue
    
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [ ]:
for season in SEASONS:
    await scrape_season(season)

Timeout error on https://www.basketball-reference.com/leagues/NBA_2024_games.html
2023-24 NBA Schedule | Basketball-Reference.com
/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/standings/NBA_2024_games-october.html
/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/standings/NBA_2024_games-november.html
/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/standings/NBA_2024_games-december.html
/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/standings/NBA_2024_games-january.html
/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/standings/NBA_2024_games-february.html
/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/standings/NBA_2024_games-march.html
/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/HTML/standings/NBA_2024_games-april.html
Timeout error on https://www.basketball-reference.com/leagues/NBA_2024_games-april.html
Timeout error on https://www.basketball-reference.com/leagues/NBA_2024_games

In [64]:
standings_files = os.listdir(STANDINGS_DIR)

In [65]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]
    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [66]:
standings_files = [s for s in standings_files if ".html" in s and "2024" in s] 
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)
    print(f)
    await scrape_game(filepath)

NBA_2024_games-march.html
NBA_2024_games-november.html
NBA_2024_games-december.html
NBA_2024_games-october.html
NBA_2024_games-february.html
NBA_2024_games-january.html
NBA_2024_games-april.html
